# Introduction

This notebook shows how to configure docarray to reuse an exsiting schema.

# Imports

In [1]:
from docarray import DocumentArray
from docarray import Document
from transformers import AutoModel, AutoTokenizer
from pprint import pprint
from weaviate import Client


/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Setup

In [2]:
# get the classnames from the weaviate instance
def get_classnames(client):
    classnames = []
    for schema in client.schema.get()['classes']:
        classnames.append(schema['class'])
    return classnames

Instantiate a Weaviate client and clear the database.

In [3]:
client = Client("http://weaviate:8080")
client.schema.delete_all()

# Multiple Instances Of Document Array

## Default Settings

Import some documents:

In [4]:
da1 = DocumentArray(storage='weaviate', config={'host': 'weaviate'})

with da1:
    da1.extend(
        [
            Document(text='Persist Documents with Weaviate.'),
            Document(text='And enjoy fast nearest neighbor search.'),
            Document(text='All while using DocArray API.'),
        ]
    )


See what classes we have in the database:

In [5]:
get_classnames(client)

['Class4abf944024fa42308900466a996f13c4',
 'Class4abf944024fa42308900466a996f13c4Meta']

Import another set of documents and see what class they are assigned to:

In [6]:
da2 = DocumentArray(storage='weaviate', config={'host': 'weaviate'})

with da2:
    da2.extend(
        [
            Document(text='lorem ipsum'),
            Document(text='dolor sit amet'),
            Document(text='consectetur adipiscing elit'),
        ]
    )

In [7]:
get_classnames(client)

['Class4abf944024fa42308900466a996f13c4',
 'Class4abf944024fa42308900466a996f13c4Meta',
 'Class619fdcbee45742648969bec5d4f8cfe9',
 'Class619fdcbee45742648969bec5d4f8cfe9Meta']

Conclusion:

Each instance of a document array is assigned to a new class.

## Reuse Class

In [8]:
client.schema.delete_all()

In [9]:
class_name = "Document"

In [10]:

da1 = DocumentArray(storage='weaviate', config={'host': 'weaviate', 'name':class_name})

with da1:
    da1.extend(
        [
            Document(text='Persist Documents with Weaviate.'),
            Document(text='And enjoy fast nearest neighbor search.'),
            Document(text='All while using DocArray API.'),
        ]
    )

In [11]:
get_classnames(client)

['Document', 'DocumentMeta']

In [12]:
da2 = DocumentArray(storage='weaviate', config={'host': 'weaviate', 'name':class_name})

with da2:
    da2.extend(
        [
            Document(text='lorem ipsum'),
            Document(text='dolor sit amet'),
            Document(text='consectetur adipiscing elit'),
        ]
    )

/usr/local/lib/python3.9/site-packages/weaviate/data/crud_data.py:662: DeprecationWarning: Weaviate Server version >= 1.14.x STRONGLY recommends using class namespaced APIs, please specify the `class_name` argument for this. The non-class namespaced APIs (None value for `class_name`) are going to be removed in the future versions of the Weaviate Server and Weaviate Python Client.
  warnings.warn(


In [13]:
get_classnames(client)

['Document', 'DocumentMeta']

There should only be 6 Document objects and 1 DocumentMeta object in the database:

In [14]:
assert client.data_object.get(class_name="Document")["totalResults"] == 6
assert client.data_object.get(class_name="DocumentMeta")["totalResults"] == 1

There should be 6 offset2ids in the DocumentMeta object:

In [15]:
meta = client.data_object.get(class_name="DocumentMeta")["objects"][0]

assert len(meta["properties"]["_offset2ids"]) == 6

Conclusion:

Instantiating a DocumentArray with a `name` key in the `config` parameter will reuse the same class